In [1]:
from keras.layers import Lambda, Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

Using TensorFlow backend.


In [11]:
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [12]:
# network parameters
input_shape = (100, 100, 3)
latent_dim = 6

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)

z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(1352)(latent_inputs)
x = Reshape((13, 13, 8))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
outputs = Conv2D(3, (3, 3), activation='relu', padding='same')(x)

# x = Dense(intermediate_dim, activation='relu')(latent_inputs)
# outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 100, 100, 16) 448         encoder_input[0][0]              
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 50, 50, 16)   0           conv2d_22[0][0]                  
__________________________________________________________________________________________________
conv2d_23 (Conv2D)              (None, 50, 50, 8)    1160        max_pooling2d_10[0][0]           
__________________________________________________________________________________________________
max_poolin

In [13]:
def vae_loss(y_true, y_pred):
    reconstruction_loss = binary_crossentropy(K.flatten(y_true), K.flatten(y_pred))
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    loss = K.mean(reconstruction_loss + kl_loss)
    return loss

vae.compile(optimizer='adam', loss=vae_loss)
vae.summary()

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'C:\\Users\\ttanj\\UoB\\WS18\\DBM\\data',
        target_size=(100, 100),
        batch_size=50,
        class_mode='input',
        shuffle=False
)

vae.fit_generator(train_generator, epochs=10, steps_per_epoch=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 100, 100, 3)       0         
_________________________________________________________________
encoder (Model)              [(None, 6), (None, 6), (N 18428     
_________________________________________________________________
decoder (Model)              (None, 100, 100, 3)       12235     
Total params: 30,663
Trainable params: 30,663
Non-trainable params: 0
_________________________________________________________________
Found 500 images belonging to 10 classes.
Epoch 1/10
10/10 [==============================] - 18s 2s/step - loss: 4.6920
Epoch 2/10
10/10 [==============================] - 16s 2s/step - loss: 1.9444
Epoch 3/10
10/10 [==============================] - 16s 2s/step - loss: 1.0812
Epoch 4/10
10/10 [==============================] - 16s 2s/step - loss: 0.4660
Epoch 5/10
10/10 [==========================